# Learning of Process Representations Using Recurrent Neural Networks

In [1]:
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering

## Load event log

In [2]:
# event log configuration
event_log_path = '../logs/'
file_name = 'huge_500_10_20_5_1_1-0.0-1.json.gz'

case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

# load file
event_log = EventLog(file_name, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
event_log.load(event_log_path + '/' + file_name, False)
event_log.preprocess()

## Representation Learning

In [3]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### Autoencoder

In [4]:
# get sequences from event log as one-hot feature vector
sequences = event_log.event_attributes_flat_onehot_features_2d

# init and train autoencoder
autoencoder = AutoencoderRepresentation(event_log)
autoencoder.build_model(sequences.shape[1], encoder_dim=vector_size)
autoencoder.fit(batch_size=n_batch_size, epochs=n_epochs, verbose=True)

Epoch 1/10
8/8 [==============================] - 0s 16ms/step - loss: 0.2497
Epoch 2/10
8/8 [==============================] - 0s 16ms/step - loss: 0.2477
Epoch 3/10
8/8 [==============================] - 0s 20ms/step - loss: 0.2451
Epoch 4/10
8/8 [==============================] - 0s 24ms/step - loss: 0.2418
Epoch 5/10
8/8 [==============================] - 0s 17ms/step - loss: 0.2370
Epoch 6/10
8/8 [==============================] - 0s 20ms/step - loss: 0.2304
Epoch 7/10
8/8 [==============================] - 0s 18ms/step - loss: 0.2210
Epoch 8/10
8/8 [==============================] - 0s 53ms/step - loss: 0.2095
Epoch 9/10
8/8 [==============================] - 0s 33ms/step - loss: 0.1946
Epoch 10/10
8/8 [==============================] - 0s 41ms/step - loss: 0.1756


In [5]:
# get feature vector
feature_vector = autoencoder.predict()

### Clustering

In [6]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [7]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 0.561019904750317
Normalized Mutual Information: 0.6772203995233733
F1-BCubed: 0.69852544657527
